In [1]:
ls

Analysis.ipynb                    format_apple_data.ipynb
Data/                             format_the_data.py
EDA.ipynb                         load_the_data.py
README.md                         preprocessing.ipynb
__pycache__/                      requirements.txt
check_if_smart_alarm_works.ipynb  run_ols.ipynb
feature_engineering.py


In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import os

# Load and parse the XML file
current_path = os.getcwd()
tree = ET.parse(current_path + '/Data/export.xml')  # Replace with the path to your XML file
root = tree.getroot()

# Extract data into a list of dictionaries
data = []
for record in root.findall('Record'):
    record_data = record.attrib  # Extract attributes of the Record tag
    for metadata in record.findall('MetadataEntry'):
        record_data[metadata.get('key')] = metadata.get('value')
    data.append(record_data)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955583 entries, 0 to 955582
Data columns (total 24 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   type                                 955583 non-null  object
 1   sourceName                           955583 non-null  object
 2   sourceVersion                        955580 non-null  object
 3   unit                                 937988 non-null  object
 4   creationDate                         955583 non-null  object
 5   startDate                            955583 non-null  object
 6   endDate                              955583 non-null  object
 7   value                                954864 non-null  object
 8   device                               921345 non-null  object
 9   HKMetadataKeyHeartRateMotionContext  141330 non-null  object
 10  HKMetadataKeySyncVersion             1728 non-null    object
 11  HKMetadataKeySyncIdentifie

In [4]:
df['type'] = df['type'].str.replace('HKQuantityTypeIdentifier', '')
df['type'] = df['type'].str.replace('HKCategoryTypeIdentifier', '')
for element in df['type'].unique():
  print(element)


BodyMassIndex
Height
BodyMass
HeartRate
OxygenSaturation
RespiratoryRate
StepCount
DistanceWalkingRunning
BasalEnergyBurned
ActiveEnergyBurned
FlightsClimbed
DietaryFatTotal
DietaryFatPolyunsaturated
DietaryFatMonounsaturated
DietaryFatSaturated
DietaryCholesterol
DietarySodium
DietaryCarbohydrates
DietaryFiber
DietarySugar
DietaryEnergyConsumed
DietaryProtein
DietaryVitaminC
DietaryCalcium
DietaryIron
DietaryPotassium
AppleExerciseTime
DistanceCycling
RestingHeartRate
VO2Max
WalkingHeartRateAverage
EnvironmentalAudioExposure
HeadphoneAudioExposure
WalkingDoubleSupportPercentage
SixMinuteWalkTestDistance
AppleStandTime
WalkingSpeed
WalkingStepLength
WalkingAsymmetryPercentage
StairAscentSpeed
StairDescentSpeed
HKDataTypeSleepDurationGoal
AppleWalkingSteadiness
EnvironmentalSoundReduction
SleepAnalysis
AppleStandHour
MindfulSession
HighHeartRateEvent
AudioExposureEvent
ToothbrushingEvent
HeadphoneAudioExposureEvent
HandwashingEvent
HeartRateVariabilitySDNN


In [5]:
df['type'].value_counts()

type
ActiveEnergyBurned                277319
HeartRate                         141356
BasalEnergyBurned                 125157
DistanceWalkingRunning            101545
StepCount                          88990
WalkingStepLength                  29755
WalkingSpeed                       29754
HeadphoneAudioExposure             25371
WalkingDoubleSupportPercentage     25001
FlightsClimbed                     16207
AppleStandTime                     14132
AppleExerciseTime                  12994
SleepAnalysis                      11221
RespiratoryRate                    10764
EnvironmentalAudioExposure         10525
WalkingAsymmetryPercentage          9903
AppleStandHour                      5374
OxygenSaturation                    4502
EnvironmentalSoundReduction         3726
StairDescentSpeed                   3578
StairAscentSpeed                    3030
HeartRateVariabilitySDNN            2579
DistanceCycling                      811
HandwashingEvent                     639
RestingHear

In [7]:
# Filter for SleepAnalysis and HeartRate
df_filtered = df[df['type'].isin(['SleepAnalysis', 'HeartRate'])]

In [8]:
df['creationDate'] = pd.to_datetime(df['creationDate'])

# Filter for the date range
start_date = '2023-09-30'
end_date = '2023-10-28'
df_filtered = df_filtered[(df_filtered['creationDate'] >= start_date) & (df_filtered['creationDate'] <= end_date)]

df['startDate'] = pd.to_datetime(df['startDate'])
df['endDate'] = pd.to_datetime(df['endDate'])


## Make heart rate dataframe

In [9]:
heart_rate_df = df_filtered[df_filtered['type'] == 'HeartRate']
heart_rate_df = heart_rate_df[['creationDate', 'startDate', 'endDate', 'value']]
heart_rate_df.head()

,creationDate,startDate,endDate,value
121528,2023-09-30 00:00:46 +0100,2023-09-29 23:49:49 +0100,2023-09-29 23:49:49 +0100,72
121529,2023-09-30 00:00:46 +0100,2023-09-29 23:54:35 +0100,2023-09-29 23:54:35 +0100,69
121530,2023-09-30 00:12:15 +0100,2023-09-29 23:59:35 +0100,2023-09-29 23:59:35 +0100,70
121531,2023-09-30 00:12:15 +0100,2023-09-30 00:04:34 +0100,2023-09-30 00:04:34 +0100,70
121532,2023-09-30 00:14:57 +0100,2023-09-30 00:09:45 +0100,2023-09-30 00:09:45 +0100,69


## Make sleep dataframe

In [10]:
sleep_analysis_df = df_filtered[df_filtered['type'] == 'SleepAnalysis']
sleep_analysis_df['value'].value_counts()

value
HKCategoryValueSleepAnalysisAsleepCore    428
HKCategoryValueSleepAnalysisInBed         212
HKCategoryValueSleepAnalysisAwake         166
HKCategoryValueSleepAnalysisAsleepREM     147
HKCategoryValueSleepAnalysisAsleepDeep    127
Name: count, dtype: int64

In [11]:
sleep_analysis_df.head()

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device,HKMetadataKeyHeartRateMotionContext,...,Maaltijd,HKVO2MaxTestType,HKDateOfEarliestDataUsedForEstimate,HKMetadataKeyAppleDeviceCalibrated,HKMetadataKeyDevicePlacementSide,HKTimeZone,HKHeartRateEventThreshold,HKMetadataKeyAudioExposureLevel,HKMetadataKeyAudioExposureDuration,HKAlgorithmVersion
945154,SleepAnalysis,iPhoner,16.3.1,NaN,2023-09-30 07:21:30 +0100,2023-09-29 21:17:54 +0100,2023-09-29 23:01:17 +0100,HKCategoryValueSleepAnalysisInBed,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe/Amsterdam,NaN,NaN,NaN,NaN
945155,SleepAnalysis,iPhoner,16.3.1,NaN,2023-09-30 07:21:30 +0100,2023-09-29 23:06:46 +0100,2023-09-29 23:08:11 +0100,HKCategoryValueSleepAnalysisInBed,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe/Amsterdam,NaN,NaN,NaN,NaN
945156,SleepAnalysis,iPhoner,16.3.1,NaN,2023-09-30 07:21:30 +0100,2023-09-29 23:13:02 +0100,2023-09-29 23:13:05 +0100,HKCategoryValueSleepAnalysisInBed,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe/Amsterdam,NaN,NaN,NaN,NaN
945157,SleepAnalysis,Apple Watch van Roman,9.1,NaN,2023-09-30 07:21:30 +0100,2023-09-29 23:40:13 +0100,2023-09-29 23:44:43 +0100,HKCategoryValueSleepAnalysisAsleepCore,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe/Amsterdam,NaN,NaN,NaN,NaN
945158,SleepAnalysis,Apple Watch van Roman,9.1,NaN,2023-09-30 07:21:30 +0100,2023-09-29 23:44:43 +0100,2023-09-29 23:45:13 +0100,HKCategoryValueSleepAnalysisAwake,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Europe/Amsterdam,NaN,NaN,NaN,NaN


In [12]:
# get first row of sleep analysis
firstrow = sleep_analysis_df.iloc[0]
firstrow['creationDate']

'2023-09-30 07:21:30 +0100'

In [18]:
# Convert startDate, endDate, and creationDate to datetime if not already done
sleep_analysis_df['startDate'] = pd.to_datetime(sleep_analysis_df['startDate'])
sleep_analysis_df['endDate'] = pd.to_datetime(sleep_analysis_df['endDate'])
sleep_analysis_df['creationDate'] = pd.to_datetime(sleep_analysis_df['creationDate'])

# Filter out 'HKCategoryValueSleepAnalysisInBed'
sleep_analysis_df = sleep_analysis_df[sleep_analysis_df['value'] != 'HKCategoryValueSleepAnalysisInBed']

# Define a function to categorize sleep states
def categorize_sleep_state(value):
    if 'Deep' in value:
        return 4
    elif 'REM' in value:
        return 2
    elif 'Core' in value:  # Assuming 'Core' is light sleep
        return 3
    elif 'Awake' in value:
        return 1
    else:
        return 'unknown'

# Categorize each record
sleep_analysis_df['sleep_state'] = sleep_analysis_df['value'].apply(categorize_sleep_state)

# Calculate the duration for each record in minutes
sleep_analysis_df['duration'] = (sleep_analysis_df['endDate'] - sleep_analysis_df['startDate']).dt.total_seconds() / 60

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device,HKMetadataKeyHeartRateMotionContext,...,HKDateOfEarliestDataUsedForEstimate,HKMetadataKeyAppleDeviceCalibrated,HKMetadataKeyDevicePlacementSide,HKTimeZone,HKHeartRateEventThreshold,HKMetadataKeyAudioExposureLevel,HKMetadataKeyAudioExposureDuration,HKAlgorithmVersion,sleep_state,duration
945157,SleepAnalysis,Apple Watch van Roman,9.1,NaN,2023-09-30 07:21:30+01:00,2023-09-29 23:40:13+01:00,2023-09-29 23:44:43+01:00,HKCategoryValueSleepAnalysisAsleepCore,NaN,NaN,...,NaN,NaN,NaN,Europe/Amsterdam,NaN,NaN,NaN,NaN,3,4.5


In [21]:
aggregated_data = []

# Process each day's data
for creation_date, day_data in sleep_analysis_df.groupby('creationDate'):
    # Ensure day_data is sorted by startDate to get the correct last record
    day_data_sorted = day_data.sort_values(by='startDate')

    # Fetch the wake-up time from the last record's creationDate
    wake_up_date = day_data_sorted['creationDate'].iloc[-1].date()
    wake_up_time = day_data_sorted['creationDate'].iloc[-1].time()

    # Calculate the total duration for each sleep state
    sleep_durations = day_data_sorted.groupby('sleep_state')['duration'].sum()

    day_summary = {
        'creation_date': creation_date,
        'wake_up_date': wake_up_date, 
        'wake_up_time': wake_up_time,
        'last_sleep_state': day_data_sorted['sleep_state'].iloc[-1],
        'sleep_deep_duration': sleep_durations.get(4, 0),  
        'sleep_light_duration': sleep_durations.get(3, 0),  
        'sleep_rem_duration': sleep_durations.get(2, 0),  
        'sleep_awake_duration': sleep_durations.get(1, 0), 
        'total_sleep_duration': sum(sleep_durations)
    }
    aggregated_data.append(day_summary)

# Convert the list of dictionaries to a DataFrame
sleep_summary_df = pd.DataFrame(aggregated_data)


In [22]:
sleep_summary_df.head()

,creation_date,wake_up_date,wake_up_time,last_sleep_state,sleep_deep_duration,sleep_light_duration,sleep_rem_duration,sleep_awake_duration,total_sleep_duration
0,2023-09-30 07:21:30+01:00,2023-09-30,07:21:30,3,74.5,295.0,81.0,7.5,458.0
1,2023-10-01 06:30:32+01:00,2023-10-01,06:30:32,3,56.5,298.0,98.0,8.5,461.0
2,2023-10-02 05:40:27+01:00,2023-10-02,05:40:27,2,48.0,296.0,103.5,9.0,456.5
3,2023-10-03 05:31:14+01:00,2023-10-03,05:31:14,3,51.0,324.0,96.5,9.0,480.5
4,2023-10-04 05:30:42+01:00,2023-10-04,05:30:42,3,60.0,293.0,115.5,11.0,479.5


In [23]:
#writing to csv
sleep_summary_df.to_csv('Data/sleep_apple.csv', index=False)
heart_rate_df.to_csv('Data/heart_rate_apple.csv', index=False)